## CBoW মডেল প্রশিক্ষণ

এই নোটবুকটি [AI for Beginners Curriculum](http://aka.ms/ai-beginners)-এর একটি অংশ।

এই উদাহরণে, আমরা CBoW ভাষা মডেল প্রশিক্ষণ করার মাধ্যমে আমাদের নিজস্ব Word2Vec এম্বেডিং স্পেস তৈরি করার প্রক্রিয়া দেখব। আমরা AG News ডেটাসেটকে টেক্সটের উৎস হিসেবে ব্যবহার করব।


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

প্রথমে চলুন আমাদের ডেটাসেট লোড করি এবং টোকেনাইজার ও শব্দভাণ্ডার সংজ্ঞায়িত করি। আমরা `vocab_size` 5000 সেট করব যাতে গণনা কিছুটা সীমিত করা যায়।


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW মডেল

CBoW $2N$ প্রতিবেশী শব্দের ভিত্তিতে একটি শব্দ অনুমান করতে শেখে। উদাহরণস্বরূপ, যখন $N=1$, তখন আমরা বাক্য *I like to train networks* থেকে নিম্নলিখিত জোড়াগুলি পাব: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks)। এখানে, প্রথম শব্দটি প্রতিবেশী শব্দ যা ইনপুট হিসেবে ব্যবহৃত হয়, এবং দ্বিতীয় শব্দটি হলো যেটি আমরা অনুমান করছি।

পরবর্তী শব্দ অনুমান করার জন্য একটি নেটওয়ার্ক তৈরি করতে, আমাদের প্রতিবেশী শব্দ ইনপুট হিসেবে সরবরাহ করতে হবে এবং শব্দের সংখ্যা আউটপুট হিসেবে পেতে হবে। CBoW নেটওয়ার্কের স্থাপত্য নিম্নরূপ:

* ইনপুট শব্দটি এমবেডিং লেয়ারের মধ্য দিয়ে পাঠানো হয়। এই এমবেডিং লেয়ারটি আমাদের Word2Vec এমবেডিং হবে, তাই আমরা এটিকে আলাদাভাবে `embedder` ভেরিয়েবল হিসেবে সংজ্ঞায়িত করব। এই উদাহরণে আমরা এমবেডিং সাইজ = 30 ব্যবহার করব, যদিও আপনি উচ্চতর মাত্রা নিয়ে পরীক্ষা করতে পারেন (বাস্তব Word2Vec-এ এটি 300)।
* এমবেডিং ভেক্টরটি এরপর একটি লিনিয়ার লেয়ারের মধ্য দিয়ে যাবে যা আউটপুট শব্দ অনুমান করবে। সুতরাং এতে `vocab_size` নিউরন থাকবে।

আউটপুটের জন্য, যদি আমরা `CrossEntropyLoss` লস ফাংশন ব্যবহার করি, তাহলে আমাদের কেবল শব্দের সংখ্যা প্রত্যাশিত ফলাফল হিসেবে সরবরাহ করতে হবে, এক-হট এনকোডিং ছাড়াই।


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## প্রশিক্ষণের ডেটা প্রস্তুত করা

এখন আমরা প্রধান ফাংশনটি প্রোগ্রাম করব যা টেক্সট থেকে CBoW শব্দ জোড়া গণনা করবে। এই ফাংশনটি আমাদের উইন্ডো সাইজ নির্ধারণ করতে দেবে এবং একটি জোড়ার সেট - ইনপুট এবং আউটপুট শব্দ ফেরত দেবে। মনে রাখবেন, এই ফাংশনটি শব্দের উপর যেমন কাজ করতে পারে, তেমনি ভেক্টর/টেনসরের উপরও কাজ করতে পারে - যা আমাদের টেক্সট এনকোড করতে দেবে, `to_cbow` ফাংশনে পাঠানোর আগে।


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


চলুন প্রশিক্ষণের ডেটাসেট প্রস্তুত করি। আমরা সমস্ত খবরের মধ্য দিয়ে যাব, `to_cbow` কল করব শব্দ জোড়ার তালিকা পেতে, এবং সেই জোড়াগুলো `X` এবং `Y` তে যোগ করব। সময় বাঁচানোর জন্য, আমরা শুধুমাত্র প্রথম ১০ হাজার খবর বিবেচনা করব - আপনি সহজেই এই সীমাবদ্ধতা সরিয়ে ফেলতে পারেন যদি আপনার অপেক্ষা করার জন্য বেশি সময় থাকে এবং আরও ভালো এম্বেডিং পেতে চান :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

আমরা সেই ডেটাকে এক ডেটাসেটে রূপান্তর করব এবং ডেটালোডার তৈরি করব:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

আমরা সেই ডেটাকে এক ডেটাসেটে রূপান্তর করব এবং ডেটালোডার তৈরি করব:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

এখন আসল প্রশিক্ষণ শুরু করা যাক। আমরা `SGD` অপ্টিমাইজার ব্যবহার করব উচ্চ লার্নিং রেট সহ। আপনি `Adam` এর মতো অন্যান্য অপ্টিমাইজার নিয়ে পরীক্ষা করতে পারেন। আমরা শুরুতে ১০ ইপোকের জন্য প্রশিক্ষণ করব - এবং যদি আপনি আরও কম ক্ষতি চান তবে এই সেলটি পুনরায় চালাতে পারেন।


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Word2Vec চেষ্টা করা

Word2Vec ব্যবহার করতে, চলুন আমাদের শব্দভাণ্ডারের সমস্ত শব্দের জন্য ভেক্টর বের করি:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

চলুন দেখি, উদাহরণস্বরূপ, কীভাবে **Paris** শব্দটি একটি ভেক্টরে এনকোড করা হয়:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Word2Vec ব্যবহার করে সমার্থক শব্দ খুঁজে বের করা বেশ আকর্ষণীয়। নিম্নলিখিত ফাংশনটি একটি প্রদত্ত ইনপুটের জন্য `n` টি নিকটতম শব্দ ফেরত দেবে। এগুলো খুঁজে বের করার জন্য আমরা $|w_i - v|$ এর নর্ম গণনা করি, যেখানে $v$ আমাদের ইনপুট শব্দের সাথে সম্পর্কিত ভেক্টর এবং $w_i$ শব্দভাণ্ডারের $i$-তম শব্দের এনকোডিং। এরপর আমরা অ্যারে সাজাই এবং `argsort` ব্যবহার করে সংশ্লিষ্ট সূচকগুলি ফেরত দিই, এবং তালিকার প্রথম `n` উপাদান গ্রহণ করি, যা শব্দভাণ্ডারে নিকটতম শব্দগুলির অবস্থান এনকোড করে।


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## মূল কথা

CBoW-এর মতো চতুর কৌশল ব্যবহার করে আমরা Word2Vec মডেল প্রশিক্ষণ দিতে পারি। আপনি চাইলে skip-gram মডেলও প্রশিক্ষণ দিতে পারেন, যা কেন্দ্রীয় শব্দটি দেওয়া হলে পার্শ্ববর্তী শব্দটি পূর্বাভাস দিতে শেখে, এবং এটি কতটা ভালো কাজ করে তা দেখতে পারেন।



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
